In [1]:
import sympy as sp
import numpy as np
import time
from numpy.polynomial import polynomial as p
import matplotlib.pyplot as plt
from sage.all import *
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler
import itertools as it

In [4]:
class RLWE:
    
    #Parameters n and q
    degree = None
    modulus = None

    
    def __init__(self, degree = 512, modulus = 4099):
        self.degree = degree
        self.modulus = modulus
        
    def polyadd(self, p1, p2):
        return np.int64(np.array(np.polyadd(p1, p2)) % self.modulus)
    
    def get_mult_matrix(a, modulus):
        n = len(a)
        A = np.array([a]).T
        for _ in range(n-1):
            a = np.append(-a[-1], a[:-1])
            A = np.concatenate((A, np.array([a]).T), axis=1) % modulus
        return A
    
    def polymul(self, p1, p2):
        p1 = np.pad(p1, (0, self.degree - len(p1)), 'constant')
        p2 = np.pad(p2, (0, self.degree - len(p2)), 'constant')
        return np.int64(np.array(np.matmul(get_mult_matrix(p1, self.modulus), p2) % self.modulus))
    
    def polydot(self, v1, v2):
        assert len(v1) == len(v2)
        res = [0]*self.degree
        for i in range(len(v1)):
            res = self.polyadd(res, self.polymul(v1[i], v2[i]))
        return res
    
    def polymatvectormul(self, A, b):
        assert np.shape(A)[1] == np.shape(b)[0]
        return np.int64(np.array([self.polydot(a, b) for a in A]))
        
    def polymatmul(self, A, B):
        assert np.shape(A)[1] == np.shape(B)[0]
        res = [self.polymatvectormul(A, B[:, 0])]
        for i in range (np.shape(B)[1]-1):
            print(res)
            res = np.vstack((res, [self.polymatvectormul(A, B[:, i+1])]))
        print(res)
        return res
    
    
    def norm(self, p1, aux = 2):
        return np.linalg.norm(p1, aux)

    

In [5]:
rlwe = RLWE(2**2, 7)

In [6]:
p = [[4, 1, 0, 6], [0, 2, 4, 2]]
q = [[3, 5, 2, 6], [2, 0, 6, 3]]
rlwe.polydot(p, q)

array([2, 5, 0, 4])

In [7]:
P = [[[4, 1, 0, 6], [0, 2, 4, 2]],
    [[0, 5, 1, 5], [2, 2, 6, 1]]]
b = [[3, 5, 2, 6], [2, 0, 6, 3]]
rlwe.polymatvectormul(P, b)

array([[2, 5, 0, 4],
       [3, 0, 5, 1]])

In [8]:
A = np.array([[[1,2,3],[4,5,6]],
    [[1,2,5],[1,4,2]]])
B = np.array([[[1,2,3],[1,2,3],[1,2,3]],
    [[1,4,2],[1,2,5],[1,4,2]]])
np.vstack(([B[:, 0]], [B[:, 1]], [B[:, 2]]))

array([[[1, 2, 3],
        [1, 4, 2]],

       [[1, 2, 3],
        [1, 2, 5]],

       [[1, 2, 3],
        [1, 4, 2]]])

In [9]:
rlwe.polymatmul(A, B)

[array([[5, 4, 2, 4],
       [4, 5, 4, 4]])]
[[[5 4 2 4]
  [4 5 4 4]]

 [[1 3 4 0]
  [5 3 6 5]]]
[[[5 4 2 4]
  [4 5 4 4]]

 [[1 3 4 0]
  [5 3 6 5]]

 [[5 4 2 4]
  [4 5 4 4]]]


array([[[5, 4, 2, 4],
        [4, 5, 4, 4]],

       [[1, 3, 4, 0],
        [5, 3, 6, 5]],

       [[5, 4, 2, 4],
        [4, 5, 4, 4]]])

In [2]:
n = 4
q = 7
P, y = PolynomialRing(IntegerModRing(q), 'y').objgen()
Rq = P.quotient(y**n + 1, 'x')
Rq

Univariate Quotient Polynomial Ring in x over Ring of integers modulo 7 with modulus y^4 + 1

In [3]:
A = np.array([Rq.random_element() for _ in range(4)]).reshape(2,2)
A

array([[2*x^3 + 4*x^2 + 3, 5*x^3 + x],
       [4*x^3 + 2*x^2 + 4*x + 2, 5*x^3]], dtype=object)

In [4]:
B = np.array([Rq.random_element() for _ in range(2)])
B

array([4*x^3 + 6*x^2 + 3, x^3 + x + 3], dtype=object)

In [13]:
np.matmul(A,B)

array([5*x^3 + 4*x^2 + 5*x + 1, 6*x^3 + 2*x^2 + 5*x + 6], dtype=object)

In [14]:
v = Rq.random_element()
w = Rq.random_element()
v, w

(6*x^3 + 3*x^2 + 6*x + 3, 6*x^3 + x^2 + 4*x + 2)

In [114]:
vv = np.array([1, 2, 0, 1])
vw = np.array([6, 0, 4, 4])
rlwe.polyadd(vv, vw), rlwe.polymul(vv, vw)

(array([0, 2, 4, 5]), array([5, 1, 0, 4]))

In [193]:
rlwe.polymatmul(np.array([[vv,vw], 
                 [rlwe.polyadd(vv, vw), rlwe.polymul(vv, vw)]]),
                      np.array([[vv, vw],
                       [vw, vv]]))

[array([[3, 0, 0, 1],
       [0, 2, 0, 5]])]
[[[3 0 0 1]
  [0 2 0 5]]

 [[3 2 0 1]
  [0 1 2 5]]]


array([[[3, 0, 0, 1],
        [0, 2, 0, 5]],

       [[3, 2, 0, 1],
        [0, 1, 2, 5]]])

In [99]:
print([v, w], [v+w, v*w])
np.matmul([[v, w], [v+w, v*w]], [[v, w],[w,v]])

[x^3 + 2*x + 1, 4*x^3 + 4*x^2 + 6] [5*x^3 + 4*x^2 + 2*x, 4*x^3 + x + 5]


array([[x^3 + 3, x^3 + 2*x + 3],
       [5*x^3 + 2*x, 5*x^3 + 2*x^2 + x]], dtype=object)

In [213]:

from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler

RingLWE(8, 1031, Discrete Gaussian sampler for polynomials of degree < 4 with σ=2.952014 in each component, x^4 + 1, 'noise', 12)

array([[[159, 423, 391, 799],
        [821, 923, 689, 60]],

       [[338, 91, 505, 540],
        [426, 881, 279, 644]],

       [[327, 294, 847, 307],
        [929, 984, 365, 419]],

       [[624, 220, 838, 872],
        [437, 190, 897, 960]],

       [[991, 330, 640, 821],
        [308, 163, 972, 982]]], dtype=object)

In [2]:
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler

In [3]:
n = 2
q = 11
P, x = PolynomialRing(ZZ, 'x').objgen()

In [4]:
#Reduce a polynomial p in Z[X] to R = Z[X]/(X^n + 1)
def R(p):
    if len(p.variables()) == 0: return p #p is a constant
    else: return p%(p.variables()[0]**n+1) if not isinstance(p, np.ndarray) else np.vectorize(R)(p)

In [5]:
#Reduce a polynomial p in R or Z[X] to Rq = R/qR = Z_q[X]/(X^n + 1)
def Rq(p):
    return (R(p)).map_coefficients(lambda x: x%q if x%q <= (q-1)/2 else (x%q)-q) if not isinstance(p, np.ndarray) else np.vectorize(Rq)(p) 

In [6]:
a = P.random_element(degree = 2*n-1)

In [7]:
def Rq_uniform_element():
    return Rq(P([randint(0, q) for _ in range(n)]))

def Rq_uniform(shape):
    return np.array([Rq_uniform_element() for _ in range(prod(shape))]).reshape(shape)

In [8]:
A = Rq_uniform((2,2))
B = Rq_uniform((2,2))
A, B, Rq(A@B)

(array([[-2*x - 4, -5],
        [2*x, -4*x + 3]], dtype=object), array([[-x + 1, -x + 1],
        [3*x + 3, 3*x + 1]], dtype=object), array([[-2*x + 1, -2*x],
        [-x + 1, -4*x - 5]], dtype=object))

In [9]:
D = DiscreteGaussianDistributionPolynomialSampler(P, n, 500)

In [10]:
def norm2(A):
    return A.norm(2) if not isinstance(A, np.ndarray) else  np.linalg.norm(np.array([norm2(a) for a in A]))

In [13]:
a = Rq_uniform((2,2))
b = Rq_uniform((2,2))
c = Rq_uniform((2,2))

In [14]:
a,b, a@b, Rq(a@b)

(array([[2*x + 5, x - 2],
        [5*x - 5, -5*x - 5]], dtype=object), array([[3*x - 2, -2*x],
        [5*x - 3, -1]], dtype=object), array([[11*x^2 - 2*x - 4, -4*x^2 - 11*x + 2],
        [-10*x^2 - 35*x + 25, -10*x^2 + 15*x + 5]], dtype=object), array([[-2*x - 4, -5],
        [-2*x + 2, 4*x + 4]], dtype=object))

In [101]:
def polyeval(p, x):
    part_sum = np.zeros_like(p[0])
    pow_x = np.eye(x.shape[0], dtype = int)
    for coef in p:
        part_sum += pow_x @ coef
        pow_x = pow_x @ x
    return part_sum

In [113]:
P = np.array([np.random.randint(0, 3, (4, 5)) for _ in range(5)], dtype = np.int64)

In [115]:
polyeval(P, np.eye(4, dtype = int))

array([[8, 4, 2, 9, 3],
       [5, 3, 7, 7, 4],
       [6, 5, 5, 2, 6],
       [8, 4, 2, 4, 7]])

array([[[1, 1, 0, 1, 0],
        [2, 2, 0, 0, 2],
        [0, 1, 2, 0, 1],
        [2, 1, 0, 1, 2]],

       [[2, 1, 1, 2, 1],
        [1, 0, 2, 2, 0],
        [1, 0, 0, 0, 2],
        [2, 1, 0, 0, 2]],

       [[1, 1, 0, 2, 1],
        [2, 1, 2, 1, 0],
        [2, 0, 0, 1, 2],
        [2, 1, 2, 1, 2]],

       [[2, 0, 0, 2, 1],
        [0, 0, 2, 2, 0],
        [1, 2, 2, 1, 1],
        [1, 0, 0, 0, 1]],

       [[2, 1, 1, 2, 0],
        [0, 0, 1, 2, 2],
        [2, 2, 1, 0, 0],
        [1, 1, 0, 2, 0]]])

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])